# Delta Lake Basic Operations

In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0b7f4d20-3a6c-47c8-9d7a-561719a460ca;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 96ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     

## Create Delta table

In [4]:
spark.sql(
    """
  CREATE TABLE fun_delta (id INT, data STRING) USING delta
"""
)

23/07/18 09:44:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[]

In [13]:
spark.sql(
    """ALTER TABLE `fun_delta` SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5')"""
)

DataFrame[]

In [5]:
spark.sql("INSERT INTO fun_delta VALUES (1, 'a'), (2, 'b'), (3, 'c')")

DataFrame[]

## Append rows to existing table

In [6]:
spark.sql("INSERT INTO fun_delta VALUES (4, 'd')")

DataFrame[]

In [7]:
spark.sql("select * from fun_delta").show()

+---+----+
| id|data|
+---+----+
|  2|   b|
|  4|   d|
|  3|   c|
|  1|   a|
+---+----+



## Alter table ADD COLUMN

In [9]:
spark.sql(
    """
ALTER TABLE fun_delta
ADD COLUMNS (
    my_new_column string
  )
"""
)

DataFrame[]

In [10]:
spark.sql("select * from fun_delta").show()

+---+----+-------------+
| id|data|my_new_column|
+---+----+-------------+
|  2|   b|         null|
|  4|   d|         null|
|  3|   c|         null|
|  1|   a|         null|
+---+----+-------------+



## Alter table RENAME COLUMN

In [14]:
spark.sql(
    """
ALTER TABLE fun_delta RENAME COLUMN data TO letter
"""
)

DataFrame[]

In [15]:
spark.sql("select * from fun_delta").show()

+---+------+-------------+
| id|letter|my_new_column|
+---+------+-------------+
|  2|     b|         null|
|  4|     d|         null|
|  3|     c|         null|
|  1|     a|         null|
+---+------+-------------+

